<a href="https://colab.research.google.com/github/TitikshaLaad/ExtionInternshipProjects/blob/main/TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
! pip install kaggle

In [45]:
#configure the pah of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [46]:
#API to fetch the dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [47]:
# extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the dependencies

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import warnings
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [49]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
#printing stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [51]:
#loading data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [52]:
#checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [53]:
#printing the first five rows from the dataframe
twitter_data.head()


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [54]:
#naming then column and reading the dataset again
colum_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=colum_names)

In [55]:
#printing the first five rows from the dataframe
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [56]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [57]:
#checking the distribution of the target column
twitter_data.target.value_counts()


,count
target,
0,800000
4,800000


Convert the target "4" to "1"

In [58]:
twitter_data['target'] = twitter_data['target'].replace(4,1)

In [59]:
#checking the distribution of the target column
twitter_data.target.value_counts()

,count
target,
0,800000
1,800000


0 ----> NEGATIVE TWEET

1 ----> POSITIVE TWEET

**Stemming**

Stemming  is the process of reducing a word to its keyword

eg: actor,actress,acting all have "act" as their root word


In [60]:
port_stem = PorterStemmer()

In [61]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content



In [62]:
twitter_data['text'] = twitter_data['text'].apply(stemming)

In [63]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behav mad see


In [64]:
print(twitter_data['text'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: text, Length: 1600000, dtype: object


In [65]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [66]:
#Separating the data and label
X = twitter_data['text'].values
Y = twitter_data['target'].values

In [67]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [68]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting this data into taining data and testing data

In [69]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [70]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [71]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [72]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [73]:
print (Y.shape,Y_train.shape,Y_test.shape)

(1600000,) (1280000,) (320000,)


In [74]:
print(Y_train)


[1 1 0 ... 1 1 0]


In [75]:
print(Y_test)

[0 1 0 ... 1 0 1]


In [76]:
#Converting the textual data to numerical data
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [77]:
print(X_train)

  (0, 436713)	1
  (0, 354543)	1
  (0, 185193)	1
  (0, 109306)	1
  (0, 235045)	1
  (0, 443066)	1
  (1, 160636)	1
  (2, 109306)	2
  (2, 124484)	1
  (2, 407301)	1
  (2, 129411)	1
  (2, 406399)	2
  (2, 433560)	1
  (2, 77929)	1
  (2, 443430)	1
  (2, 266729)	1
  (2, 409143)	1
  (2, 178061)	1
  (2, 150715)	1
  (2, 132311)	1
  (2, 288470)	1
  (3, 406399)	1
  (3, 158711)	1
  (3, 151770)	1
  (3, 56476)	1
  :	:
  (1279996, 318303)	1
  (1279996, 434014)	1
  (1279996, 390130)	1
  (1279996, 373144)	1
  (1279996, 238077)	1
  (1279996, 238078)	1
  (1279997, 5685)	1
  (1279997, 273084)	1
  (1279997, 112591)	1
  (1279998, 412553)	1
  (1279998, 93795)	1
  (1279998, 169461)	1
  (1279998, 124765)	1
  (1279998, 435463)	1
  (1279998, 153281)	1
  (1279998, 156297)	1
  (1279998, 162047)	1
  (1279998, 275288)	1
  (1279998, 385313)	1
  (1279999, 242268)	1
  (1279999, 31410)	1
  (1279999, 435572)	1
  (1279999, 433612)	1
  (1279999, 135384)	1
  (1279999, 96224)	1


In [78]:
print(X_test)

  (0, 15110)	1
  (0, 31168)	1
  (0, 67828)	1
  (0, 106069)	1
  (0, 132364)	1
  (0, 138164)	1
  (0, 171378)	1
  (0, 271016)	1
  (0, 279082)	1
  (0, 388348)	1
  (0, 398906)	1
  (0, 409143)	2
  (0, 420984)	1
  (1, 6463)	1
  (1, 15110)	1
  (1, 145393)	1
  (1, 217562)	1
  (1, 256777)	1
  (1, 348135)	1
  (1, 366203)	1
  (2, 22532)	1
  (2, 34401)	1
  (2, 89448)	1
  (2, 183312)	1
  (2, 256834)	1
  :	:
  (319994, 443794)	1
  (319995, 107868)	1
  (319995, 109379)	1
  (319995, 155493)	1
  (319995, 213324)	1
  (319995, 232891)	1
  (319995, 296662)	1
  (319995, 315813)	1
  (319995, 324496)	1
  (319995, 416257)	1
  (319995, 420984)	1
  (319995, 444934)	1
  (319996, 397506)	1
  (319996, 438709)	1
  (319997, 98792)	1
  (319997, 169411)	1
  (319997, 261286)	1
  (319997, 288421)	1
  (319997, 349904)	1
  (319997, 416695)	1
  (319997, 444770)	1
  (319998, 130192)	1
  (319998, 438748)	1
  (319999, 389755)	1
  (319999, 400636)	1


**Training the machine learning model**



Logistic Regression Model

In [79]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import io
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import preprocessing
plt.rc("font", size = 14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
sns.set(style="white")
sns.set(style="whitegrid", color_codes = True)

In [80]:
model = LogisticRegression(max_iter=1000)




In [81]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy score

In [82]:
#accuray score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [83]:
print('Accuracy score on training data :',training_data_accuracy)

Accuracy score on training data : 0.8255703125


In [84]:
#accuray score on training data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [85]:
print('Accuracy score on test data :',testing_data_accuracy)

Accuracy score on test data : 0.7755375


Model accuracy =77.8%

Saving the trained modl

In [87]:
import pickle

In [88]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the Saved model for future predictions.

In [89]:
#loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [100]:

X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print(' Negative Tweet')
else:
  print(' Positive Tweet')



0
[0]
 Negative Tweet


In [102]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print(' Negative Tweet')
else:
  print(' Positive Tweet')

1
[1]
 Positive Tweet
